In [2]:
import os
import numpy as np
import phonemizer
import librosa.display
import matplotlib.pyplot as plt
import tensorflow as tf
import keras as keras
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
import nltk
from nltk.tokenize import word_tokenize
from tqdm import tqdm

nltk.download('punkt')  # Download the necessary data for tokenization


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\soumy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')


if gpus:
    for gpu in gpus:
        print(gpu.name)
        tf.config.experimental.set_memory_growth(gpu, True)


tf.config.functions_run_eagerly()


/physical_device:GPU:0


False

In [4]:
import csv

# Function to load text and audio paths
def load_data_paths(dataset_path):
    text_paths = []
    audio_paths = []
    
    for root, _, files in os.walk(dataset_path):
        for file in files:
            if file.endswith(".csv"):
                text_paths.append(os.path.join(root, file))
            elif file.endswith(".wav"):
                audio_paths.append(os.path.join(root, file))
    
    return text_paths, audio_paths

# Load text from text paths
def load_texts(text_paths):
    texts = []
    for text_path in text_paths:
        with open(text_path, 'r', encoding='utf-8') as csvfile:
            for line in csvfile:
                fields = line.strip().split('|')
                if len(fields) >= 3:  # Ensure there are at least 3 fields
                    sentence = fields[2]
                    texts.append(sentence)

    return texts

# Preprocess text
path_to_ljspeech = 'LJSpeech-1.1'
text_paths, audio_paths = load_data_paths(path_to_ljspeech)
texts = load_texts(text_paths)

In [5]:
def preprocess_text(texts):
    token_texts = []
    print("Preprocessing text")
    for text in texts:
        tokenized_texts = word_tokenize(text)
        #print(tokenized_texts)
        token_texts.append(tokenized_texts)
    return token_texts


preprocessed_texts=preprocess_text(texts)


Preprocessing text


In [6]:
  
# Preprocess audio
def preprocess_audio(audio_paths, sample_rate=5000, n_mels=120):
    print("Preprocessing audio")
    mel_specs = []
    for path in audio_paths:
        audio, _ = librosa.load(path, sr=sample_rate)
        
        mel_spec = librosa.feature.melspectrogram(y=audio, sr=sample_rate, n_mels=n_mels)
        mel_specs.append(mel_spec)
    return mel_specs

# Load and preprocess LJSpeech data

preprocessed_mel_specs = preprocess_audio(audio_paths)

Preprocessing audio


In [7]:

print(len(preprocessed_mel_specs))
print(len(preprocessed_texts))

13100
13100


In [8]:
import gc
gc.collect()

3322

In [9]:
def convert_to_db(mel_specs):
    return 10 * np.log10(mel_specs + 1e-9)

preprocessed_mel_specs_db = [convert_to_db(mel_spec) for mel_spec in preprocessed_mel_specs]

# Split data into train and validation sets
train_size = int(0.8 * len(preprocessed_texts))
train_mel_specs, val_mel_specs = preprocessed_mel_specs_db[:train_size], preprocessed_mel_specs_db[:train_size]

print("Length of preprocessed_mel_specs_db:", len(preprocessed_mel_specs_db))
print("Train size:", train_size)
print("Length of train_mel_specs:", len(train_mel_specs))
print("Length of val_mel_specs:", len(val_mel_specs))

# Find the maximum number of frames among all mel spectrograms
max_frames = max(spec.shape[1] for spec in train_mel_specs)

# Pad or truncate mel spectrograms to have the same number of frames
padded_train_mel_specs = [np.pad(spec, ((0, 0), (0, max_frames - spec.shape[1])), mode='constant') for spec in train_mel_specs]
padded_val_mel_specs = [np.pad(spec, ((0, 0), (0, max_frames - spec.shape[1])), mode='constant') for spec in val_mel_specs]

# Define the number of frames (time steps) and number of mel bins
n_frames = padded_train_mel_specs[0].shape[1]
n_mels = padded_train_mel_specs[0].shape[0]



Length of preprocessed_mel_specs_db: 13100
Train size: 10480
Length of train_mel_specs: 10480
Length of val_mel_specs: 10480


In [10]:

# LSTM Model
model = Sequential([
    LSTM(units=256, return_sequences=True, input_shape=(n_frames, n_mels)),
    LSTM(units=256, return_sequences=True),
    Dense(units=n_mels)
])

optimizer = Adam(learning_rate=0.001)
loss_fn = MeanSquaredError()

# Compile the model
model.compile(optimizer=optimizer, loss=loss_fn)

# Reshape the input mel spectrograms
reshaped_train_mel_specs = np.array(padded_train_mel_specs).reshape(-1, n_frames, n_mels)
reshaped_val_mel_specs = np.array(padded_val_mel_specs).reshape(-1, n_frames, n_mels)

# Define the number of epochs and batch size
epochs = 1000
batch_size = 32

# Fit the model on training data
model.fit(reshaped_train_mel_specs, reshaped_train_mel_specs,
          validation_data=(reshaped_val_mel_specs, reshaped_val_mel_specs),
          batch_size=batch_size,
          epochs=epochs)

# Save the trained model
model.save('tts_model3.h5')

Epoch 1/500
328/328 [==============================] - 19s 41ms/step - loss: 203.5782 - val_loss: 161.0510
Epoch 2/500
328/328 [==============================] - 12s 37ms/step - loss: 142.2600 - val_loss: 128.7411
Epoch 3/500
328/328 [==============================] - 12s 38ms/step - loss: 119.5164 - val_loss: 110.3064
Epoch 4/500
328/328 [==============================] - 13s 39ms/step - loss: 102.0462 - val_loss: 94.0597
Epoch 5/500
328/328 [==============================] - 13s 39ms/step - loss: 87.2385 - val_loss: 80.7019
Epoch 6/500
328/328 [==============================] - 14s 43ms/step - loss: 74.9687 - val_loss: 69.4600
Epoch 7/500
328/328 [==============================] - 14s 41ms/step - loss: 64.5778 - val_loss: 59.8777
Epoch 8/500
328/328 [==============================] - 13s 41ms/step - loss: 56.0295 - val_loss: 52.2134
Epoch 9/500
328/328 [==============================] - 14s 42ms/step - loss: 49.0938 - val_loss: 45.9353
Epoch 10/500
328/328 [==========================